In [1]:
from helper import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()

In [2]:
import nest_asyncio
nest_asyncio.apply()

Define a Simple Tool

In [3]:
from llama_index.core.tools import FunctionTool

def add(x: int, y: int) -> int:
    """Adds two integers together."""
    return x + y

def mystery(x: int, y: int) -> int: 
    """Mystery function that operates on top of two numbers."""
    return (x + y) * (x + y)


add_tool = FunctionTool.from_defaults(fn=add)
mystery_tool = FunctionTool.from_defaults(fn=mystery)

In [4]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")
response = llm.predict_and_call(
    [add_tool, mystery_tool], 
    "Tell me the output of the mystery function on 2 and 9", 
    verbose=True
)
print(str(response))

=== Calling Function ===
Calling function: mystery with args: {"x": 2, "y": 9}
=== Function Output ===
121
121


Define an Auto-Retrieval Tool

In [8]:
from llama_index.core import SimpleDirectoryReader
# load documents
documents = SimpleDirectoryReader(input_files=["ing_industrial.pdf"]).load_data()

In [10]:
from llama_index.core.node_parser import SentenceSplitter
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [11]:
print(nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: ing_industrial.pdf
file_path: ing_industrial.pdf
file_type: application/pdf
file_size: 372978
creation_date: 2024-06-21
last_modified_date: 2024-06-18

Grado en
Ingeniería Industrial
Sobre el grado
Con este grado, tu potencial te llevará a diseñar 
todos los procesos de una empresa, inﬂuir en todas sus áreas y sacar lo mejor de cada colaborador y colaboradora utilizando síntesis de la ingeniería y el diseño para evaluar y mejorar los resultados. Esta ingeniería busca instruirte en el equipamiento, energía, materiales y procesos de producción de alta calidad y servicios útiles con alta consideración al medio ambiente.
Dónde podrás trabajar
Con
trol de calidad
Asegura el cumplimiento de todas las características y herramientas de productos y servicios.
Procesos y manufactura
Mejora los procesos y propón estrategias sustentables de crecimiento.
Producción
Implementa estrategias de optimización para conseguir el rendimiento máximo de procesos industriales.
Logístic

In [12]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(similarity_top_k=2)

In [13]:
from llama_index.core.vector_stores import MetadataFilters

query_engine = vector_index.as_query_engine(
    similarity_top_k=2,
    filters=MetadataFilters.from_dicts(
        [
            {"key": "page_label", "value": "1"}
        ]
    )
)

response = query_engine.query(
    "Where i will work whit this profession?", 
)

In [14]:
print(str(response))

You will work in areas such as quality control, processes and manufacturing, production, and logistics with this profession.


In [15]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '1', 'file_name': 'ing_industrial.pdf', 'file_path': 'ing_industrial.pdf', 'file_type': 'application/pdf', 'file_size': 372978, 'creation_date': '2024-06-21', 'last_modified_date': '2024-06-18'}


Define the Auto-Retrieval Tool

In [16]:
from typing import List
from llama_index.core.vector_stores import FilterCondition


def vector_query(
    query: str, 
    page_numbers: List[str]
) -> str:
    """Perform a vector search over an index.
    
    query (str): the string query to be embedded.
    page_numbers (List[str]): Filter by set of pages. Leave BLANK if we want to perform a vector search
        over all pages. Otherwise, filter by the set of specified pages.
    
    """

    metadata_dicts = [
        {"key": "page_label", "value": p} for p in page_numbers
    ]
    
    query_engine = vector_index.as_query_engine(
        similarity_top_k=2,
        filters=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
        )
    )
    response = query_engine.query(query)
    return response
    

vector_query_tool = FunctionTool.from_defaults(
    name="vector_tool",
    fn=vector_query
)

In [18]:
llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
response = llm.predict_and_call(
    [vector_query_tool], 
    "Why i should study in Utel as described on page 3?", 
    verbose=True
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "Why should I study in Utel?", "page_numbers": ["3"]}
=== Function Output ===
You should study in Utel because of the recognized academic quality, the opportunity to graduate with a professional degree, the chance to showcase your knowledge in future job opportunities, the global community that enriches your education, the 24/7 virtual campus allowing you to create your own schedule, the quality credentials from partnerships with renowned institutions, the guidance and support from expert tutors throughout your education, the investment in yourself with real, current, and transformative training at an accessible cost, and the official recognition of studies from various accreditation bodies.


In [19]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '3', 'file_name': 'ing_industrial.pdf', 'file_path': 'ing_industrial.pdf', 'file_type': 'application/pdf', 'file_size': 372978, 'creation_date': '2024-06-21', 'last_modified_date': '2024-06-18'}


Let's add some other tools

In [20]:
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool

summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description=(
        "Useful if you want to get a summary of ing_industrial"
    ),
)

In [21]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool], 
    "What are the Academic Validity described on page 2?", 
    verbose=True
)

=== Calling Function ===
Calling function: summary_tool with args: {"input": "Academic Validity on page 2"}
=== Function Output ===
The academic validity of the program is recognized by being incorporated into the National Educational System, with an official title granted by the Mexican Ministry of Education. Additionally, graduates of the Industrial Engineering program from Utel can obtain academic equivalence in the United States as a "Bachelor of Science in Industrial Engineering earned through distance education" through specific agencies affiliated with NACES or AICE.


In [22]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '1', 'file_name': 'ing_industrial.pdf', 'file_path': 'ing_industrial.pdf', 'file_type': 'application/pdf', 'file_size': 372978, 'creation_date': '2024-06-21', 'last_modified_date': '2024-06-18'}
{'page_label': '2', 'file_name': 'ing_industrial.pdf', 'file_path': 'ing_industrial.pdf', 'file_type': 'application/pdf', 'file_size': 372978, 'creation_date': '2024-06-21', 'last_modified_date': '2024-06-18'}
{'page_label': '3', 'file_name': 'ing_industrial.pdf', 'file_path': 'ing_industrial.pdf', 'file_type': 'application/pdf', 'file_size': 372978, 'creation_date': '2024-06-21', 'last_modified_date': '2024-06-18'}


In [23]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool], 
    "What is a summary of the paper?", 
    verbose=True
)

=== Calling Function ===
Calling function: summary_tool with args: {"input": "ing_industrial"}
=== Function Output ===
The Bachelor of Science in Industrial Engineering program offered by Utel provides students with the knowledge and skills to design processes within a company, influence various areas, and optimize the performance of collaborators. Graduates can work in quality control, process improvement, production optimization, and logistics management. The program covers strategic business planning, business process alignment, control systems development, and process management. Additionally, the degree is officially recognized in Mexico and can be equivalent to a Bachelor of Science in Industrial Engineering in the United States. Students can specialize in areas such as administration, information technology, and finance, and the curriculum includes subjects like industry structure, algebra, physics, statistics, engineering methods, and quality control. The program offers a flexi